In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time


video_path = "./istockphoto-482522882-mp4-480x480-is.mp4"

cap = cv2.VideoCapture(video_path)

if (cap.isOpened()== False):
  print("Error opening video stream or file")

In [43]:


while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:

    cv2.imshow('Frame',frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  else:
    break

cap.release()

cv2.destroyAllWindows()

In [45]:

cap = cv2.VideoCapture(video_path)

if (cap.isOpened() == False):
  print("Error opening video stream or file")

first_frame = True
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if first_frame:
            background = gray
            first_frame = False
            continue
        
        img_diff = np.abs(gray.astype("float32") - background).astype("uint8")
        
        background = gray
    
        cv2.imshow('Frame', img_diff)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()

cv2.destroyAllWindows()

In [8]:

BUFFER_SIZE = 5
first_frame = True

cap = cv2.VideoCapture(video_path)
if (cap.isOpened()== False):
  print("Error opening video stream or file")

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if first_frame:
            h, w, c = frame.shape
            background_buffer = np.ones((BUFFER_SIZE, h, w), dtype=np.uint8) * np.expand_dims(gray, axis=0)
            first_frame = False
            continue

        background = background_buffer.mean(axis=0).astype("float32")

        background_buffer = np.roll(background_buffer, shift=1, axis=0)
        background_buffer[0] = gray

        gray = np.abs(gray.astype("float32") - background).astype("uint8")


        cv2.imshow('Frame', frame)
        cv2.imshow('Background', background)
        cv2.imshow('Subtracted Background', gray)
        cv2.imshow('Mask', (255*(gray>10)).astype("uint8"))
        time.sleep(0.5)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
      break

cap.release()

cv2.destroyAllWindows()

In [4]:
# B = (1 − α)B(x, y,t − 1) + αI(x, y,t)

BUFFER_SIZE = 5
ALPHA = 0.5
first_frame = True

cap = cv2.VideoCapture(video_path)
# cap = cv2.VideoCapture(0) - Use laptop webcam
if (cap.isOpened()== False):
  print("Error opening video stream or file")

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if first_frame:
            h, w, c = frame.shape
            background = gray.copy()
            first_frame = False
            continue

        background = (1 - ALPHA) * background.astype("float32") + ALPHA * gray.astype("float32")

        gray = np.abs(gray.astype("float32") - background).astype("uint8")


        cv2.imshow('Frame', frame)
        cv2.imshow('Background', background)
        cv2.imshow('Subtracted Background', gray)
        cv2.imshow('Mask', (255*(gray>10)).astype("uint8"))
        time.sleep(0.1)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
      break

cap.release()

cv2.destroyAllWindows()

In [4]:
 
ALPHA = 0.3 
 

first_frame = True

while True:
    ret, frame = cap.read()
    
    if not ret:
        break  

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if first_frame:
        background = gray.copy().astype("float32")
        first_frame = False
    else:
        background = (1 - ALPHA) * background + ALPHA * gray.astype("float32")

cap.release()

background_image = background.astype("uint8")  
cv2.imwrite("background_final.png", background_image)  

True

In [4]:
# Sem fundo
ALPHA = 0.02 
first_frame = True
background = None
 

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if first_frame:
        background = gray.copy().astype("float32")
        first_frame = False
    else:
        background = (1 - ALPHA) * background + ALPHA * gray.astype("float32")

    background_display = cv2.normalize(background, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    cv2.imshow('Background Estimado', background_display)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
ALPHA = 0.02  
first_frame = True
 

background = None
first_frame = True

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    if first_frame:
        background = gray.copy().astype("float32")
        first_frame = False
        continue
    else:
        background = (1 - ALPHA) * background + ALPHA * gray.astype("float32")

    diff = cv2.absdiff(gray, cv2.convertScaleAbs(background))
    _, mask = cv2.threshold(diff, 10, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)
        if area > 500:  # Filtra por área mínima para evitar ruído
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Detecção de Movimento com Bounding Box', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()